In [1]:
import numpy as np
import pandas as pd
import mlflow

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')

import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_predict, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN

import optuna
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sudhirjoon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sudhirjoon/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/opt/anaconda3/lib/python3.11/site-packages/dask/dataframe/__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [5]:
mlflow.set_tracking_uri("http://13.60.79.0:5000")
mlflow.set_experiment("Exp 6 - Detailed HPT of XGboost")

2024/12/30 15:13:12 INFO mlflow.tracking.fluent: Experiment with name 'Exp 6 - Detailed HPT of XGboost' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://capstone-yt-mlflow-bucket/717286515719630891', creation_time=1735567992877, experiment_id='717286515719630891', last_update_time=1735567992877, lifecycle_stage='active', name='Exp 6 - Detailed HPT of XGboost', tags={}>

In [6]:
df = pd.read_csv('reddit_preprocessing.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [9]:
# Step 1: Remap the class labels from [-1, 0, 1] to [2, 0, 1]
df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

ngram_range = (1, 2)  # Bigram setting
max_features = 1000  # Set max_features to 1000 for TF-IDF

# Step 4: Train-test split before vectorization and resampling
X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

# Step 2: Vectorization using TF-IDF, fit on training data only
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train_vec = vectorizer.fit_transform(X_train)  # Fit on training data
X_test_vec = vectorizer.transform(X_test)  # Transform test data

# Step 3: Imbalance dataset using ADASYN
adasyn = ADASYN(random_state=42)
X_train_vec, y_train = adasyn.fit_resample(X_train_vec, y_train)


# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_ADASYN_TFIDF_Bigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for XGBoost
def objective_xgboost(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 0.2),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 7),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0),
        'random_state': 42
    }

    model = XGBClassifier(**params)
     
    # Log trial in MLflow
    with mlflow.start_run(nested=True):  # Nested allows Optuna trials to appear under a single experiment
        # Log trial number in MLflow
        mlflow.set_tag("mlflow.runName", f"XGB_Trial_{trial.number}")
        # Log trial parameters
        for param_name, param_value in params.items():
            mlflow.log_param(param_name, param_value)

        # Train and evaluate the model
        model.fit(X_train_vec, y_train)
        y_pred = model.predict(X_test_vec)
        accuracy = accuracy_score(y_test, y_pred)

        # Log trial metrics
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("trial_number", trial.number)

    # Return the metric for Optuna optimization
    return accuracy


# Step 7: Run Optuna for XGBoost, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_xgboost, n_trials=50)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = XGBClassifier(
        n_estimators=best_params['n_estimators'],
        max_depth=best_params['max_depth'],
        learning_rate=best_params['learning_rate'],
        min_child_weight=best_params['min_child_weight'],
        subsample=best_params['subsample'],
        colsample_bytree=best_params['colsample_bytree'],
        gamma=best_params['gamma'],
        reg_alpha=best_params['reg_alpha'],
        reg_lambda=best_params['reg_lambda'],
        random_state=42
    )

    # Log the best model with MLflow, passing the algo_name as "xgboost"
    log_mlflow("XGBoost", best_model, X_train_vec, X_test_vec, y_train, y_test)

# Run the experiment for XGBoost
run_optuna_experiment()


[I 2024-12-30 17:12:31,230] A new study created in memory with name: no-name-86bf33de-5d7c-45fa-8155-c7ba7cf5da87
[I 2024-12-30 17:12:48,536] Trial 0 finished with value: 0.8861516804504663 and parameters: {'n_estimators': 459, 'max_depth': 8, 'learning_rate': 0.03473579516692396, 'min_child_weight': 5, 'subsample': 0.8854880480680802, 'colsample_bytree': 0.915293421182102, 'gamma': 0.12087953413191507, 'reg_alpha': 0.9679521809590507, 'reg_lambda': 0.4678693768186719}. Best is trial 0 with value: 0.8861516804504663.


🏃 View run XGB_Trial_0 at: http://13.60.79.0:5000/#/experiments/717286515719630891/runs/f731451bf7c54a889abe0512c1497c39
🧪 View experiment at: http://13.60.79.0:5000/#/experiments/717286515719630891


[I 2024-12-30 17:12:58,706] Trial 1 finished with value: 0.8989970086222066 and parameters: {'n_estimators': 230, 'max_depth': 7, 'learning_rate': 0.15029411109105711, 'min_child_weight': 4, 'subsample': 0.8048954024341375, 'colsample_bytree': 0.9337650718012495, 'gamma': 0.1476344158522324, 'reg_alpha': 0.09035227834822597, 'reg_lambda': 0.7137065673027699}. Best is trial 1 with value: 0.8989970086222066.


🏃 View run XGB_Trial_1 at: http://13.60.79.0:5000/#/experiments/717286515719630891/runs/bd9eb0e06e3c41bc86883e2953a16b8f
🧪 View experiment at: http://13.60.79.0:5000/#/experiments/717286515719630891


[I 2024-12-30 17:13:06,863] Trial 2 finished with value: 0.8402252331515044 and parameters: {'n_estimators': 315, 'max_depth': 5, 'learning_rate': 0.028454343945158184, 'min_child_weight': 5, 'subsample': 0.8813301257352621, 'colsample_bytree': 0.8677784863384903, 'gamma': 0.5856571323061787, 'reg_alpha': 0.3855980492923203, 'reg_lambda': 0.6601427001742962}. Best is trial 1 with value: 0.8989970086222066.


...

🏃 View run XGB_Trial_48 at: http://13.60.79.0:5000/#/experiments/717286515719630891/runs/fa324fe8216b432e80c1169db76f84c5
🧪 View experiment at: http://13.60.79.0:5000/#/experiments/717286515719630891


[I 2024-12-30 17:20:37,896] Trial 49 finished with value: 0.907091325004399 and parameters: {'n_estimators': 267, 'max_depth': 6, 'learning_rate': 0.18779856116920668, 'min_child_weight': 2, 'subsample': 0.7993530839476384, 'colsample_bytree': 0.6762670388056441, 'gamma': 0.5119258999221035, 'reg_alpha': 0.4307342241387505, 'reg_lambda': 0.4444037679100022}. Best is trial 30 with value: 0.9079711420024635.


🏃 View run XGB_Trial_49 at: http://13.60.79.0:5000/#/experiments/717286515719630891/runs/709436fffe0f4284999783601ee0f2a9
🧪 View experiment at: http://13.60.79.0:5000/#/experiments/717286515719630891


2024/12/30 17:20:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBoost_ADASYN_TFIDF_Bigrams at: http://13.60.79.0:5000/#/experiments/717286515719630891/runs/7a97f3bd43d04846895bc80c3615f9e5
🧪 View experiment at: http://13.60.79.0:5000/#/experiments/717286515719630891
